# Classification in PySpark's MLlib Project Solution

### Genre classification
Now it's time to leverage what we learned in the lectures to a REAL classification project! Have you ever wondered what makes us, humans, able to tell apart two songs of different genres? How we do we inherenly know the difference between a pop song and heavy metal? This type of classifcation may seem easy for us, but it's a very difficult challenge for a computer to do. So the question is, could an automatic genre classifcation model be possible? 

For this project we will be classifying songs based on a number of characteristics into a set of 23 electronic genres. This technology could be used by an application like Pandora to recommend songs to users or just create meaningful channels. Super fun!

### Dataset
*beatsdataset.csv*
Each row is an electronic music song. The dataset contains 100 song for each genre among 23 electronic music genres, they were the top (100) songs of their genres on November 2016. The 71 columns are audio features extracted of a two random minutes sample of the file audio. These features have been extracted using pyAudioAnalysis (https://github.com/tyiannak/pyAudioAnalysis).

### Your task
Create an algorithm that classifies songs into the 23 genres provided. Test out several different models and select the highest performing one. Also play around with feature selection methods and finally try to make a recommendation to a user.  

For the feature selection aspect of this project, you may need to get a bit creative if you want to select features from a non-tree algorithm. I did not go over this aspect of PySpark intentionally in the previous lectures to give you chance to get used to researching the PySpark documentation page. Here is the link to the Feature Selectors section of the documentation that just might come in handy: https://spark.apache.org/docs/latest/ml-features.html#feature-selectors

Good luck! Have fun :)

### Source
https://www.kaggle.com/caparrini/beatsdataset

In [42]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ClassificationProject').getOrCreate()



In [43]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler

In [44]:
path ="Datasets/"
df = spark.read.csv(path+'beatsdataset.csv',inferSchema=True,header=True)

In [45]:
df.limit(5).toPandas()

,_c0,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,...,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class
0,0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,...,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom
1,1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,...,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom
2,2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,...,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom
3,3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,...,0.001591,0.003514,0.009477,0.023162,0.004165,0.015379,133.333333,0.158876,129.0,BigRoom
4,4,0.151730,0.148405,3.194498,0.293730,0.267231,1.353005,0.003872,0.292055,-21.371157,...,0.003945,0.004131,0.011330,0.028188,0.002639,0.019079,133.333333,0.190708,129.0,BigRoom


In [46]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (null

In [47]:
df.groupBy('class').count().show(100)

+--------------------+-----+
|               class|count|
+--------------------+-----+
|           PsyTrance|  100|
|           HardDance|  100|
|              Breaks|  100|
|  HardcoreHardTechno|  100|
|   IndieDanceNuDisco|  100|
|              Trance|  100|
|           DeepHouse|  100|
|ElectronicaDowntempo|  100|
|           ReggaeDub|  100|
|             Minimal|  100|
|         DrumAndBass|  100|
|             Dubstep|  100|
|             BigRoom|  100|
|              Techno|  100|
|               House|  100|
|         FutureHouse|  100|
|        ElectroHouse|  100|
|           GlitchHop|  100|
|           TechHouse|  100|
|              HipHop|  100|
|           FunkRAndB|  100|
|               Dance|  100|
|    ProgressiveHouse|  100|
+--------------------+-----+



## Format Data

In [48]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+------+---------+----------------+-------------------+-----------------+------------------+---------------+------------------+---------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+---------+------------+-------------------+----------------------+--------------------+---------------------+------------------+---------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-

In [49]:
input_columns = df.columns # Collect the column names as a list
input_columns = input_columns[1:-1] # keep only relevant columns: from column 1 to 

dependent_var = 'class'

In [50]:
renamed = df.withColumn('label_str', df[dependent_var].cast(StringType()))
indexer = StringIndexer(inputCol='label_str', outputCol='label')
indexed = indexer.fit(renamed).transform(renamed)

In [51]:
numeric_inputs = []
string_inputs = []
for column in input_columns:
    if str(indexed.schema[column].dataType) == 'StringType':
        indexer = StringIndexer(inputCol = column, outputCol=column+'_num')
        indexed = indexer.fit(indexed).transform(indexed)
        new_col_name = column+'_num'
        string_inputs.append(new_col_name)
    else:
        numeric_inputs.append(column)
indexed.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (null

#### Skewness

In [52]:
d= {}
for col in numeric_inputs:
    d[col]=indexed.approxQuantile(col,[0.01, 0.99], 0.25)

for col in numeric_inputs:
    skew = indexed.agg(skewness(indexed[col])).collect()
    skew=skew[0][0]
    if skew > 1:
        indexed = indexed.withColumn(col, \
                                    log(when(df[col] < d[col][0], d[col][0])\
                                       .when(indexed[col]> d[col][1], d[col][1])\
                                       .otherwise(indexed[col])+1).alias(col))
        print(col, " has been treated for pos skew", skew)
    elif skew < -1:
        indexed = indexed.withColumn(col, \
                                    exp(when(df[col] < d[col][0], d[col][0])\
                                       .when(indexed[col]> d[col][1], d[col][1])\
                                       .otherwise(indexed[col])+1).alias(col))
        print(col, " has been treated for neg skew", skew)

7-SpectralFluxm  has been treated for pos skew 1.6396138160129063
22-ChromaVector1m  has been treated for pos skew 2.4162415204309258
23-ChromaVector2m  has been treated for pos skew 4.154796693680583
24-ChromaVector3m  has been treated for pos skew 1.1974019617504328
25-ChromaVector4m  has been treated for pos skew 2.446635863594906
26-ChromaVector5m  has been treated for pos skew 2.154482876187508
27-ChromaVector6m  has been treated for pos skew 2.01234064472543
28-ChromaVector7m  has been treated for pos skew 1.1829228989215521
29-ChromaVector8m  has been treated for pos skew 3.7372643733999955
30-ChromaVector9m  has been treated for pos skew 2.4117416421548645
31-ChromaVector10m  has been treated for pos skew 2.1979538518563233
32-ChromaVector11m  has been treated for pos skew 2.1924295373960554
33-ChromaVector12m  has been treated for pos skew 2.278981912155668
41-SpectralFluxstd  has been treated for pos skew 1.8577721462401056
56-ChromaVector1std  has been treated for pos skew 1

In [53]:
minimums = df.select([min(c).alias(c) for c in df.columns if c in numeric_inputs])
min_array = minimums.select(array(numeric_inputs).alias('mins'))
df_minimum = min_array.select(array_min(min_array.mins)).collect()
df_minimum = df_minimum[0][0]
df_minimum

-30.3789543716

In [54]:
features_list = numeric_inputs + string_inputs
assembler = VectorAssembler(inputCols=features_list, outputCol='features')
output = assembler.transform(indexed).select('features','label')


In [55]:
output.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.136439587512,0...|  0.0|
|[0.117038518483,0...|  0.0|
|[0.0853077737447,...|  0.0|
|[0.103049917216,0...|  0.0|
|[0.151729948738,0...|  0.0|
|[0.127046737192,0...|  0.0|
|[0.123395302003,0...|  0.0|
|[0.140027382431,0...|  0.0|
|[0.117635200751,0...|  0.0|
|[0.137400181488,0...|  0.0|
|[0.148838734199,0...|  0.0|
|[0.119846749928,0...|  0.0|
|[0.0786813648231,...|  0.0|
|[0.138335144235,0...|  0.0|
|[0.101304207661,0...|  0.0|
|[0.132862180406,0...|  0.0|
|[0.1533035231,0.1...|  0.0|
|[0.118001599962,0...|  0.0|
|[0.110992493712,0...|  0.0|
|[0.124299777916,0...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [56]:
scaler = MinMaxScaler(inputCol='features', outputCol='scaledFeatures', min = 0, max=1000)
scalerModel = scaler.fit(output)
scaled_data= scalerModel.transform(output)

In [57]:
scaled_data.show()

+--------------------+-----+--------------------+
|            features|label|      scaledFeatures|
+--------------------+-----+--------------------+
|[0.136439587512,0...|  0.0|[519.818266700239...|
|[0.117038518483,0...|  0.0|[435.295463992565...|
|[0.0853077737447,...|  0.0|[297.057129121742...|
|[0.103049917216,0...|  0.0|[374.352647734368...|
|[0.151729948738,0...|  0.0|[586.432337466259...|
|[0.127046737192,0...|  0.0|[478.897323979332...|
|[0.123395302003,0...|  0.0|[462.989461602348...|
|[0.140027382431,0...|  0.0|[535.448873531282...|
|[0.117635200751,0...|  0.0|[437.894973202184...|
|[0.137400181488,0...|  0.0|[524.003195633498...|
|[0.148838734199,0...|  0.0|[573.836456502165...|
|[0.119846749928,0...|  0.0|[447.529820358620...|
|[0.0786813648231,...|  0.0|[268.188480022892...|
|[0.138335144235,0...|  0.0|[528.076459415282...|
|[0.101304207661,0...|  0.0|[366.747280005321...|
|[0.132862180406,0...|  0.0|[504.232915471732...|
|[0.1533035231,0.1...|  0.0|[593.287780078539...|


In [58]:
final_data=scaled_data.select('label', 'scaledFeatures')
final_data = final_data.withColumnRenamed('scaledFeatures', 'features')

In [59]:
train, test= final_data.randomSplit([0.7, 0.3])

In [60]:
train.count()

1578

In [61]:
test.count()

722

### Modeling

In [62]:
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.sql.functions import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [63]:
Bin_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction')
MC_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [64]:
def ClassTrainEval(classifier,features,classes, folds,train,test):

    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,train):
        
        if Mtype == "OneVsRest":
            # instantiate the base classifier.
            lr = LogisticRegression()
            # instantiate the One Vs Rest Classifier.
            OVRclassifier = OneVsRest(classifier=lr)
#             fitModel = OVRclassifier.fit(train)
            # Add parameters of your choice here:
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1, 0.01]) \
                .build()
            #Cross Validator requires the following parameters:
            crossval = CrossValidator(estimator=OVRclassifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 is best practice
            # Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
        if Mtype == "MultilayerPerceptronClassifier":
            # specify layers for the neural network:
            # input layer of size features, two intermediate of features+1 and same size as features
            # and output of size number of classes
            # Note: crossvalidator cannot be used here
            features_count = len(features[0][0])
            layers = [features_count, features_count+1, features_count, classes]
            MPC_classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
            fitModel = MPC_classifier.fit(train)
            return fitModel
        if Mtype in("LinearSVC","GBTClassifier") and classes != 2: # These classifiers currently only accept binary classification
            print(Mtype," could not be used because PySpark currently only accepts binary classification data for this algorithm")
            return
        if Mtype in("LogisticRegression","NaiveBayes","RandomForestClassifier","GBTClassifier","LinearSVC","DecisionTreeClassifier"):
  
            # Add parameters of your choice here:
            if Mtype in("LogisticRegression"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .addGrid(classifier.maxIter, [10, 15,20])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("NaiveBayes"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6]) \
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("RandomForestClassifier"):
                paramGrid = (ParamGridBuilder() \
                               .addGrid(classifier.maxDepth, [2, 5, 10])
#                                .addGrid(classifier.maxBins, [5, 10, 20])
#                                .addGrid(classifier.numTrees, [5, 20, 50])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("GBTClassifier"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
#                              .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .addGrid(classifier.maxIter, [10, 15,50,100])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("LinearSVC"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxIter, [10, 15]) \
                             .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .build())
            
            # Add parameters of your choice here:
            if Mtype in("DecisionTreeClassifier"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                             .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .build())
            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=2) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        
        if Mtype in("OneVsRest"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            # Extract list of binary models
            models = BestModel.models
            for model in models:
                print('\033[1m' + 'Intercept: '+ '\033[0m',model.intercept,'\033[1m' + '\nCoefficients:'+ '\033[0m',model.coefficients)
                coeff_array = model.coefficients.toArray()
                coeff_scores = []
                for x in coeff_array:
                    coeff_scores.append(float(x))
                result = spark.createDataFrame(zip(input_columns,coeff_scores), schema = ['feature','coeff'])
                print(result.orderBy(result['coeff'].desc()).show(truncate=False))
        if Mtype == "MultilayerPerceptronClassifier":
            print("")
            print('\033[1m' + Mtype," Weights"+ '\033[0m')
            print('\033[1m' + "Model Weights: "+ '\033[0m',fitModel.weights.size)
            print("")

        if Mtype in("DecisionTreeClassifier", "GBTClassifier","RandomForestClassifier"):
            # FEATURE IMPORTANCES
            # Estimate of the importance of each feature.
            # Each feature’s importance is the average of its importance across all trees 
            # in the ensemble The importance vector is normalized to sum to 1. 
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Feature Importances"+ '\033[0m')
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            featureImportances = BestModel.featureImportances.toArray()
            print(featureImportances)
            
            if Mtype in("DecisionTreeClassifier"):
                global DT_featureImportances
                DT_featureImportances = BestModel.featureImportances.toArray()
                global DT_BestModel
                DT_BestModel = BestModel
            if Mtype in("GBTClassifier"):
                global GBT_featureImportances
                GBT_featureImportances = BestModel.featureImportances.toArray()
                global GBT_BestModel
                GBT_BestModel = BestModel
            if Mtype in("RandomForestClassifier"):
                global RF_featureImportances
                RF_featureImportances = BestModel.featureImportances.toArray()
                global RF_BestModel
                RF_BestModel = BestModel

        if Mtype in("LogisticRegression"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Coefficient Matrix"+ '\033[0m')
            print("You should compares these relative to eachother")
            print("Coefficients: \n" + str(BestModel.coefficientMatrix))
            print("Intercept: " + str(BestModel.interceptVector))
            global LR_coefficients
            LR_coefficients = BestModel.coefficientMatrix.toArray()
            global LR_BestModel
            LR_BestModel = BestModel

        if Mtype in("LinearSVC"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")
            print("Coefficients: \n" + str(BestModel.coefficients))
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result

### Logit (Chi-Square Numfeatures=30)

In [65]:
classifier= LogisticRegression()

paramGrid = (ParamGridBuilder().addGrid(classifier.maxIter, [10, 15,20]).build())

crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=MC_evaluator,
                         numFolds=2)

fitModel = crossval.fit(train)
BestModel = fitModel.bestModel

print('Intercept: ' +str(BestModel.interceptVector))
print('Coefficients: \n' + str(BestModel.coefficientMatrix))

LR_BestModel = BestModel

predictions=fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100

print(accuracy)


Intercept: [0.006327749088245603,0.020273095974877958,0.08247165437409971,0.08622416442355908,0.12582556446955476,-0.02092723663260653,0.006835293954409371,-0.1519403123463118,-0.01810374499302218,0.034740099427522673,-0.06730159788586712,0.010501738595491077,-0.035865041966963086,-0.0696017955056091,-0.00804941095215304,-0.02194806513207668,-0.006426817940390233,-0.09374627556060107,0.03786926177207716,0.029478834094131937,-0.07403515754194856,0.12053866662936238,0.006859333654217729]
Coefficients: 
DenseMatrix([[ 5.32977819e-04,  1.92339292e-03,  1.79232188e-03, ...,
              -2.40750282e-03, -9.89613679e-04,  1.93692382e-03],
             [-3.59532315e-04,  4.57092338e-05,  5.04748462e-04, ...,
               4.56277915e-03, -3.85152745e-03,  2.14977566e-03],
             [-2.99364967e-04,  5.13046881e-04,  1.08536264e-03, ...,
              -2.20879472e-03,  3.92030817e-04, -9.65459694e-04],
             ...,
             [ 2.99706591e-04, -8.65067073e-04, -6.70341283e-04, ...

In [66]:

coeff_array = BestModel.coefficientMatrix.toArray()
coeff_scores = []
for x in coeff_array[0]:
    coeff_scores.append(float(x))

result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
result.show(100)

+--------------------+--------------------+
|             feature|               coeff|
+--------------------+--------------------+
|              1-ZCRm| 5.32977818938724E-4|
|           2-Energym|0.001923392915402...|
|    3-EnergyEntropym|0.001792321881340...|
| 4-SpectralCentroidm|8.048828244733436E-4|
|   5-SpectralSpreadm|0.001248253380924...|
|  6-SpectralEntropym|9.813579013497574E-4|
|     7-SpectralFluxm|-0.00199969199956...|
|  8-SpectralRolloffm|3.617704284047664...|
|           9-MFCCs1m|0.003554236755111...|
|          10-MFCCs2m|-0.00175323636687...|
|          11-MFCCs3m|-2.34699846001469...|
|          12-MFCCs4m|4.491271670002251...|
|          13-MFCCs5m|3.561334868151826...|
|          14-MFCCs6m|-1.64095021625326...|
|          15-MFCCs7m|-1.95132391256584...|
|          16-MFCCs8m|-3.36765825269175...|
|          17-MFCCs9m|-5.22181906404892E-4|
|         18-MFCCs10m|9.859545719092485E-4|
|         19-MFCCs11m|-1.66984746252996...|
|         20-MFCCs12m|-4.5961665

### One Vs Rest Classifier fwe

In [67]:
lr = LogisticRegression()
classifier = OneVsRest(classifier = lr)

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam,[0.1,0.01]) \
    .build()

crossval=CrossValidator(estimator=classifier,
                       estimatorParamMaps=paramGrid,
                       evaluator=MulticlassClassificationEvaluator(),
                       numFolds=2)

fitModel=crossval.fit(train)

BestModel = fitModel.bestModel

models= BestModel.models
for model in models:
    print('\033[1m' + 'Intercept: '+ '\033[0m',model.intercept,'\033[1m' + '\nCoefficients:'+ '\033[0m',model.coefficients)

predictions=fitModel.transform(test)
accuracy=(MC_evaluator.evaluate(predictions))*100
print(accuracy)

Intercept:  -5.327607821747951 
Coefficients: [-0.0001247469296608693,0.0010563147071388703,0.0016299668375744092,-0.00010697809951257754,0.001516056297451319,0.0004961902249819562,-0.0011904106180791114,-0.0005837362237384427,0.00343075931848717,-0.0011577457049325968,0.0008890539481378145,0.001437723271617106,0.0007785885950998867,-0.0009049575471640344,-0.0003810973707986484,-0.00035702901643329925,-0.0011060958574965177,0.0014083799065055845,-0.00032151704695149315,0.0002809155840371197,-0.0009324376622848985,0.001760021745283577,-0.0021234708913158615,0.0014969293147074868,-0.00028731966451902923,0.0005061172071978576,-0.0004123308598637224,-0.0024883974102714993,-0.0003038139469712204,-9.189626328586254e-05,-0.00012651095272910352,-0.00029507297698849305,-0.0003592977292747732,-0.0014474156432776083,-0.0020243104431017784,0.0008394868755785959,5.02712771162173e-05,-0.0023918972802158497,-0.0018315213146606545,2.863108186198744e-05,0.00011788693588364681,-0.0004750917735094573,0.0

Intercept:  6.356285011986955 
Coefficients: [0.0016750584230403249,0.003450131034641761,-0.0016126989996392215,-3.58068799602509e-05,-0.001076650050760353,-0.00019256575620239688,0.00231696558047221,0.00023592696515099362,-0.003382907943845887,-0.0016742660403461496,-0.00010326996506951217,-2.51163433931245e-05,0.0028015459406174332,-0.0005490271537591776,0.0016741383649346289,-0.0012419369457037372,0.00033134561014287826,-0.0012920057006437096,0.0001596496029678244,-0.00022125464788047599,0.0015110693131820065,-0.0015274927762545464,-0.0005360375891956395,0.00035033599365471625,-0.001893748810391613,-0.0032043287880247455,-0.0011387642913875387,0.0006428356760920399,0.0009175139224814714,0.0010969561326689941,-0.0008910811156966763,-0.0025181063031135486,-0.00028298587354630374,-0.0013953417890367238,0.00010375706838407185,-0.0023095659710905855,-0.0008718648457851036,0.0013369487010505444,0.00012769796629383734,-0.0013789339087752516,-0.0028737971857860363,-0.0014041186915207792,-0.

Intercept:  -2.1332264584206064 
Coefficients: [0.0007464282488642449,-0.0033040237731031107,0.001996136175051764,0.001143007614223008,0.002222433468568251,0.00020558890206725969,0.0011330194823688997,0.0004189883297367194,0.0012871799674672084,-0.0008644087041356764,-0.0010926272455598104,0.000903067346937901,-0.0010126048105342088,-0.0009578340318615835,-0.0004471244133879923,0.001558641985216794,0.00048047004850114487,-0.0007746914019089484,0.001139269242306947,0.0011753365769881098,0.0015153684763631852,0.0004347121858176374,0.0006172772332530342,-0.001194751560032499,0.0015241714845209988,-0.00012441277873991745,0.00019085995446587592,-6.378209525718598e-05,0.00028386770611870386,0.0009948724934100619,0.0016265895111172246,-0.0005328739338815845,0.0010076683980567974,-0.0009813755752253297,-0.0010222237238687874,-0.003827189388972431,-0.002500667694925711,-0.002478498669708403,-0.0021780209183043025,-0.0009188561835560869,0.0017327491312230101,-0.000849146321205932,-0.002558469480

### Multilayer Perceptron Classifier fwe

In [68]:
features = train.select(['features']).collect()
features_count = len(features[0][0])
class_count=final_data.select(countDistinct('label')).collect()
classes = class_count[0][0]

layers = [features_count, features_count+1, features_count, classes]

classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed= 1234)

fitModel = classifier.fit(train)

print('\033[1m' + "Model Weights: "+ '\033[0m',fitModel.weights.size)


predictions = fitModel.transform(test)

accuracy = (MC_evaluator.evaluate(predictions))*100

print("Accuracy: ",accuracy)

Model Weights:  12023
Accuracy:  18.144044321329638


### NaiveBayes

In [69]:
classifier= NaiveBayes()
paramGrid = (ParamGridBuilder()\
             .addGrid(classifier.smoothing, [0.0,0.2,0.4,0.6])\
             .build())
crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps = paramGrid,
                         evaluator= MulticlassClassificationEvaluator(),
                         numFolds=2)

fitModel = crossval.fit(train)

predictions = fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100
print("Accuracy: ",accuracy)

Accuracy:  40.16620498614959


### Decision Tree

In [70]:
classifier = DecisionTreeClassifier()
paramGrid = (ParamGridBuilder()\
            .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30])\
            .addGrid(classifier.maxBins, [10,20,40,80, 100])\
            .build())

crossval = CrossValidator(estimator = classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=2)

fitModel = crossval.fit(train)

BestModel = fitModel.bestModel
featureImportances= BestModel.featureImportances.toArray()

print("Feature Importances: ", featureImportances)

predictions= fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100
print("Accuracy: ", accuracy)

Feature Importances:  [0.077028   0.02745731 0.01815883 0.01406153 0.00479245 0.
 0.06583188 0.         0.05407617 0.02367465 0.01408755 0.00951782
 0.00173532 0.00341321 0.01002467 0.00459738 0.00545495 0.00886397
 0.00694405 0.00772908 0.00359039 0.00274237 0.01250895 0.0023725
 0.00594858 0.00224597 0.         0.00174741 0.         0.
 0.00685076 0.0093459  0.00183422 0.00969638 0.01520677 0.00366043
 0.0145113  0.01680881 0.02119703 0.         0.00856446 0.00891435
 0.03023958 0.03540894 0.00551505 0.01508603 0.         0.00946655
 0.00325686 0.01214712 0.00091628 0.02958179 0.01093284 0.01104048
 0.01535916 0.         0.00348096 0.00267805 0.0050712  0.00611655
 0.00793518 0.0059349  0.         0.0037584  0.00266624 0.00676977
 0.00794323 0.00617212 0.         0.03242106 0.22490626]
Accuracy:  37.257617728531855


In [71]:
imp_scores = []
for x in featureImportances:
    imp_scores.append(int(x))

result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
print(result.orderBy(result["score"].desc()).show(truncate=False))

+---------------------+-----+
|feature              |score|
+---------------------+-----+
|52-MFCCs10std        |0    |
|53-MFCCs11std        |0    |
|54-MFCCs12std        |0    |
|55-MFCCs13std        |0    |
|56-ChromaVector1std  |0    |
|57-ChromaVector2std  |0    |
|58-ChromaVector3std  |0    |
|59-ChromaVector4std  |0    |
|60-ChromaVector5std  |0    |
|61-ChromaVector6std  |0    |
|62-ChromaVector7std  |0    |
|63-ChromaVector8std  |0    |
|64-ChromaVector9std  |0    |
|65-ChromaVector10std |0    |
|66-ChromaVector11std |0    |
|67-ChromaVector12std |0    |
|68-ChromaDeviationstd|0    |
|69-BPM               |0    |
|70-BPMconf           |0    |
|71-BPMessentia       |0    |
+---------------------+-----+
only showing top 20 rows

None


### Random Forest Classifer

In [72]:
classifier = RandomForestClassifier()
paramGrid = (ParamGridBuilder()\
            .addGrid(classifier.maxDepth, [2, 5, 10])
            .addGrid(classifier.maxBins, [5, 10 , 20])
            .addGrid(classifier.numTrees,[5, 20, 50])
            .build())

crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=2)

fitModel = crossval.fit(train)

BestModel= fitModel.bestModel
featureImportances= BestModel.featureImportances.toArray()
print("Feature Importances: ", featureImportances)

predictions = fitModel.transform(test)

accuracy = (MC_evaluator.evaluate(predictions))*100

print(" ")
print("Accuracy: ", accuracy)

Feature Importances:  [0.02506208 0.02707725 0.02104294 0.01333907 0.01117727 0.0088462
 0.01551201 0.00714973 0.01783216 0.01718051 0.01278898 0.00884493
 0.00863997 0.00898261 0.00963838 0.00882911 0.00900272 0.00925924
 0.00920059 0.00981966 0.00934604 0.00991391 0.00928938 0.01510976
 0.00843184 0.00993823 0.00910101 0.00984084 0.00769268 0.0078826
 0.00858376 0.01045547 0.00958591 0.01167134 0.01162439 0.01115673
 0.01363387 0.01918384 0.02262271 0.01023284 0.01510046 0.01071707
 0.02171162 0.01881061 0.0114218  0.0139186  0.01279879 0.01794861
 0.01280647 0.0152012  0.01341866 0.01788879 0.01470956 0.01839441
 0.0174341  0.01013387 0.00853691 0.01045082 0.00991492 0.0102664
 0.00841088 0.01160771 0.00964672 0.00913043 0.00906054 0.01066449
 0.00825124 0.00825027 0.04579168 0.02488019 0.08819962]
 
Accuracy:  47.091412742382275


In [73]:
imp_scores = []
for x in featureImportances:
    imp_scores.append(float(x))
    
# Then zip with input_columns list and create a df
result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
print(result.orderBy(result["score"].desc()).show(truncate=False))

+----------------------+--------------------+
|feature               |score               |
+----------------------+--------------------+
|71-BPMessentia        |0.08819961581085413 |
|69-BPM                |0.045791676007317324|
|2-Energym             |0.027077253042407767|
|1-ZCRm                |0.025062083469539136|
|70-BPMconf            |0.02488018894380021 |
|39-SpectralSpreadstd  |0.022622713253370164|
|43-MFCCs1std          |0.02171161824923858 |
|3-EnergyEntropym      |0.021042940235161456|
|38-SpectralCentroidstd|0.019183844439266712|
|44-MFCCs2std          |0.018810613105668367|
|54-MFCCs12std         |0.018394406355297273|
|48-MFCCs6std          |0.01794860551589802 |
|52-MFCCs10std         |0.01788878922113427 |
|9-MFCCs1m             |0.01783215645769427 |
|55-MFCCs13std         |0.01743409787396502 |
|10-MFCCs2m            |0.01718051395309192 |
|7-SpectralFluxm       |0.015512013814870638|
|50-MFCCs8std          |0.015201196884346594|
|24-ChromaVector3m     |0.01510976

### Feature Selection for RFC and Logit

In [74]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

In [75]:
### code from LaylaAi's udemy course
classifiers=[OneVsRest(), LogisticRegression()]
maximum= len(input_columns)

for n in range(10, maximum, 10):
    print("Testing top n= ", n, " features")
    selector=ChiSqSelector(numTopFeatures=n, featuresCol="features", outputCol="selectedFeatures", labelCol= "label")
    bestFeaturesDf= selector.fit(final_data).transform(final_data)
    bestFeaturesDf= bestFeaturesDf.select('label', 'selectedFeatures')
    bestFeaturesDf= bestFeaturesDf.withColumnRenamed('selectedFeatures', 'features')
    features= bestFeaturesDf.select(['features']).collect()
    
    train,test= bestFeaturesDf.randomSplit([0.7, 0.3])
    
    folds= 2
    
    columns = ['Classifier', 'Result']
    vals = [('Place Holder', 'N/A')]
    results = spark.createDataFrame(vals, columns)
    
    for classifier in classifiers:
        new_result = ClassTrainEval(classifier, features, classes, folds, train, test)
        results = results.union(new_result)
        
    results = results.where('Classifier!="Place Holder"')
    results.show(100, False)
    

Testing top n=  10  features
 
OneVsRest
Intercept:  -2.0366462111133212 
Coefficients: [-0.0005752594713940964,0.00015924626506736675,-0.006098812114142482,-0.0007171696078283604,0.00031668256889342975,0.0003065116918083959,-0.0006572134733437084,0.0016194540932313766,-0.002160900549895043,0.0018958738082606656]
+-------------------+---------------------+
|feature            |coeff                |
+-------------------+---------------------+
|10-MFCCs2m         |0.0018958738082606656|
|8-SpectralRolloffm |0.0016194540932313766|
|5-SpectralSpreadm  |3.1668256889342975E-4|
|6-SpectralEntropym |3.065116918083959E-4 |
|2-Energym          |1.5924626506736675E-4|
|1-ZCRm             |-5.752594713940964E-4|
|7-SpectralFluxm    |-6.572134733437084E-4|
|4-SpectralCentroidm|-7.171696078283604E-4|
|9-MFCCs1m          |-0.002160900549895043|
|3-EnergyEntropym   |-0.006098812114142482|
+-------------------+---------------------+

None
Intercept:  -7.696526702526525 
Coefficients: [0.00183451474163

+-------------------+----------------------+
|feature            |coeff                 |
+-------------------+----------------------+
|6-SpectralEntropym |0.0018469078575391415 |
|7-SpectralFluxm    |0.0015848773534941154 |
|10-MFCCs2m         |0.0015036295874660225 |
|4-SpectralCentroidm|6.689644567145351E-4  |
|8-SpectralRolloffm |6.369260097688406E-4  |
|5-SpectralSpreadm  |1.0333856765245224E-4 |
|2-Energym          |-0.0011327512495431314|
|1-ZCRm             |-0.002496670971027062 |
|3-EnergyEntropym   |-0.0028112635120365955|
|9-MFCCs1m          |-0.003471089729144676 |
+-------------------+----------------------+

None
Intercept:  -4.364303455367623 
Coefficients: [0.002106843876834983,-0.0009413279069609462,0.0010472562588161082,0.001105462126698673,0.0022577325138016216,-0.0005642568877830937,-0.0003752404816965884,-0.0009909706749680568,0.002620664731191733,-0.00393421361637113]
+-------------------+---------------------+
|feature            |coeff                |
+-------

+-------------------+----------------------+
|feature            |coeff                 |
+-------------------+----------------------+
|10-MFCCs2m         |0.00945573467687867   |
|3-EnergyEntropym   |0.002708089324040792  |
|9-MFCCs1m          |0.002048432797760987  |
|2-Energym          |0.0013348631720791807 |
|7-SpectralFluxm    |-1.0933977644056485E-5|
|6-SpectralEntropym |-2.8517706421354056E-5|
|4-SpectralCentroidm|-0.0016558004213730413|
|8-SpectralRolloffm |-0.002627303354901193 |
|5-SpectralSpreadm  |-0.004123132554873408 |
|1-ZCRm             |-0.004879326427989962 |
+-------------------+----------------------+

None
Intercept:  -3.8660508043921684 
Coefficients: [-0.0031684606993591822,8.729347942811146e-05,0.0006757130057539147,0.005101858074925588,0.0031003485356038184,-0.0015738816933057033,0.0010403134638884634,-0.001968130688719553,0.0008932706108497055,-0.003180422902403353]
+-------------------+----------------------+
|feature            |coeff                 |
+---

+------------------+------+
|Classifier        |Result|
+------------------+------+
|OneVsRest         |25.07 |
|LogisticRegression|23.52 |
+------------------+------+

Testing top n=  20  features
 
OneVsRest
Intercept:  -2.1008225230345663 
Coefficients: [0.00286267216669078,0.0004926217703195879,-0.004954817370643486,7.630801129232283e-05,-0.0007111647274742342,-0.0008244146966446949,0.0006729042855091452,0.00087740185778574,-0.0002580826042051339,-0.0021525201729227766,-0.0027554479340011874,-0.0009153243878235367,-0.0005515557775277354,-0.0019488774500770864,0.0013853388617495008,-0.00011001545781036007,-0.0004023335414033334,0.0011744889263547039,-0.002930917929185269,0.002372773350737075]
+-------------------+----------------------+
|feature            |coeff                 |
+-------------------+----------------------+
|1-ZCRm             |0.00286267216669078   |
|20-MFCCs12m        |0.002372773350737075  |
|15-MFCCs7m         |0.0013853388617495008 |
|18-MFCCs10m        |0.00

+-------------------+----------------------+
|feature            |coeff                 |
+-------------------+----------------------+
|19-MFCCs11m        |0.0047717784673351514 |
|1-ZCRm             |0.0026704667171506234 |
|13-MFCCs5m         |0.0024086328094465475 |
|12-MFCCs4m         |0.0017833073787380084 |
|2-Energym          |0.0017057744542218597 |
|15-MFCCs7m         |0.0013805981610914679 |
|6-SpectralEntropym |0.0010613751454456601 |
|14-MFCCs6m         |8.280809874153225E-4  |
|7-SpectralFluxm    |7.270487086941188E-4  |
|5-SpectralSpreadm  |3.0865780295822524E-4 |
|20-MFCCs12m        |-5.350846174535463E-5 |
|8-SpectralRolloffm |-7.873031612597696E-4 |
|10-MFCCs2m         |-0.0010971531835923033|
|4-SpectralCentroidm|-0.0018244147762976942|
|9-MFCCs1m          |-0.001967298399720317 |
|11-MFCCs3m         |-0.0021849558681757905|
|18-MFCCs10m        |-0.002280379763475768 |
|16-MFCCs8m         |-0.0026723042551308057|
|17-MFCCs9m         |-0.0027730627711736277|
|3-EnergyE

...

+---------------------+---------------------+
|feature              |coeff                |
+---------------------+---------------------+
|9-MFCCs1m            |0.0050061609848410715|
|43-MFCCs1std         |0.0034645745677177684|
|22-ChromaVector1m    |0.002395918819804486 |
|70-BPMconf           |0.0023249397888972685|
|30-ChromaVector9m    |0.0018012387626943388|
|16-MFCCs8m           |0.0016432020737475322|
|18-MFCCs10m          |0.001554065628456601 |
|11-MFCCs3m           |0.0015457996210320496|
|2-Energym            |0.001504528175916523 |
|37-EnergyEntropystd  |0.0014658126758984801|
|51-MFCCs9std         |9.490547776473764E-4 |
|6-SpectralEntropym   |9.300931954650559E-4 |
|3-EnergyEntropym     |7.096516025843169E-4 |
|25-ChromaVector4m    |6.46376326682387E-4  |
|12-MFCCs4m           |5.975692074949871E-4 |
|24-ChromaVector3m    |5.481862337746344E-4 |
|13-MFCCs5m           |5.477623204641065E-4 |
|32-ChromaVector11m   |5.041694218966412E-4 |
|46-MFCCs4std         |4.971633887

+--------------------+---------------------+
|feature             |coeff                |
+--------------------+---------------------+
|56-ChromaVector1std |0.005361828360712423 |
|69-BPM              |0.0027816441288569587|
|9-MFCCs1m           |0.0022954748816592274|
|50-MFCCs8std        |0.002148433727313108 |
|43-MFCCs1std        |0.0019774431698532558|
|11-MFCCs3m          |0.0015669271707192817|
|66-ChromaVector11std|0.0015444069430287453|
|62-ChromaVector7std |0.0015062338804391264|
|52-MFCCs10std       |0.0013307147221093682|
|58-ChromaVector3std |0.001018875782845234 |
|27-ChromaVector6m   |0.0010047874684149857|
|59-ChromaVector4std |9.782383707297641E-4 |
|53-MFCCs11std       |9.44530511031019E-4  |
|18-MFCCs10m         |9.020330770078221E-4 |
|21-MFCCs13m         |8.616280994178366E-4 |
|45-MFCCs3std        |8.443360034529914E-4 |
|13-MFCCs5m          |8.420266752635723E-4 |
|35-ZCRstd           |7.989274281528484E-4 |
|67-ChromaVector12std|7.518558489685723E-4 |
|32-Chroma

+--------------------+---------------------+
|feature             |coeff                |
+--------------------+---------------------+
|69-BPM              |0.0036536181244617156|
|44-MFCCs2std        |0.003370939317931728 |
|55-MFCCs13std       |0.003168311323428454 |
|36-Energystd        |0.002641402140501172 |
|3-EnergyEntropym    |0.00233071131431432  |
|63-ChromaVector8std |0.00229294210343538  |
|54-MFCCs12std       |0.0020096460402373355|
|27-ChromaVector6m   |0.0019157772433192437|
|34-ChromaDeviationm |0.0016073868380801523|
|18-MFCCs10m         |0.0014860282897685608|
|52-MFCCs10std       |0.001067713103282287 |
|46-MFCCs4std        |0.0010392960736416142|
|24-ChromaVector3m   |9.640301495846889E-4 |
|48-MFCCs6std        |8.5311145464213E-4   |
|15-MFCCs7m          |8.508770602510055E-4 |
|6-SpectralEntropym  |8.127892275537794E-4 |
|66-ChromaVector11std|7.91200381798846E-4  |
|49-MFCCs7std        |7.805420502762905E-4 |
|60-ChromaVector5std |6.031792924960962E-4 |
|2-Energym

+----------------------+---------------------+
|feature               |coeff                |
+----------------------+---------------------+
|29-ChromaVector8m     |0.003499277141498636 |
|43-MFCCs1std          |0.002908352424994844 |
|41-SpectralFluxstd    |0.0028501974748288976|
|7-SpectralFluxm       |0.002687765214341369 |
|21-MFCCs13m           |0.002068158106139719 |
|38-SpectralCentroidstd|0.002048138618551188 |
|39-SpectralSpreadstd  |0.0020233199114395068|
|23-ChromaVector2m     |0.0017685593338607965|
|70-BPMconf            |0.0017570148302831327|
|45-MFCCs3std          |0.0017072178995462197|
|33-ChromaVector12m    |0.0016461424387570205|
|37-EnergyEntropystd   |0.0014637088114300763|
|34-ChromaDeviationm   |0.0014564189569386465|
|47-MFCCs5std          |0.0012680546448222412|
|32-ChromaVector11m    |0.0012014276394617556|
|13-MFCCs5m            |0.001107417561675057 |
|22-ChromaVector1m     |7.268042511052394E-4 |
|28-ChromaVector7m     |7.165908581598595E-4 |
|44-MFCCs2std

+----------------------+---------------------+
|feature               |coeff                |
+----------------------+---------------------+
|11-MFCCs3m            |0.0032193879390392892|
|67-ChromaVector12std  |0.002651313755411129 |
|38-SpectralCentroidstd|0.0016843705481038285|
|43-MFCCs1std          |0.0016533627362578026|
|35-ZCRstd             |0.0016245732059408167|
|30-ChromaVector9m     |0.0016056910529219776|
|42-SpectralRolloffstd |0.0015668082667349638|
|70-BPMconf            |0.0015350844960886432|
|15-MFCCs7m            |0.001473884067628245 |
|41-SpectralFluxstd    |0.0014538521857972516|
|28-ChromaVector7m     |0.001379334463436413 |
|17-MFCCs9m            |0.0012384641396957052|
|44-MFCCs2std          |0.0011638241353168922|
|25-ChromaVector4m     |0.001137396059634726 |
|54-MFCCs12std         |0.001044831519291648 |
|37-EnergyEntropystd   |0.0010427413195330932|
|13-MFCCs5m            |9.696612545398464E-4 |
|7-SpectralFluxm       |7.836428336003808E-4 |
|31-ChromaVec

#### Selector for Random Forest

In [76]:
from pyspark.ml.feature import VectorSlicer

In [77]:
### code from LaylaAi's udemy course
classifiers=[RandomForestClassifier()]
maximum= len(input_columns)

for n in range(10, maximum, 10):
    print("Testing top n= ", n, " features")
    
    best_n_features= featureImportances.argsort()[-n:][::-1]
    best_n_features= best_n_features.tolist()
    vs= VectorSlicer(inputCol='features', outputCol='best_features',indices=best_n_features)
    bestFeaturesDf= vs.transform(final_data)
    train,test= bestFeaturesDf.randomSplit([0.7, 0.3])
    
    folds= 2
    
    columns = ['Classifier', 'Result']
    vals = [('Place Holder', 'N/A')]
    results = spark.createDataFrame(vals, columns)
    
    for classifier in classifiers:
        new_result = ClassTrainEval(classifier, features, classes, folds, train, test)
        results = results.union(new_result)
        
    results = results.where('Classifier!="Place Holder"')
    results.show(100, False)
    

Testing top n=  10  features
 
RandomForestClassifier  Feature Importances
(Scores add up to 1)
Lowest score is the least important
 
[0.02888963 0.03108573 0.0159277  0.01337458 0.0125509  0.01025784
 0.01419054 0.00750732 0.01859048 0.01639994 0.01111623 0.01124447
 0.00998606 0.00770556 0.01305929 0.00894956 0.00583386 0.00741931
 0.01141357 0.01080727 0.00858454 0.00893249 0.00730578 0.0115212
 0.00794147 0.01098896 0.006381   0.01059559 0.01040044 0.00776125
 0.01073851 0.01119676 0.00833582 0.01017087 0.01328641 0.01185696
 0.01166108 0.02124423 0.02076403 0.01086397 0.01522136 0.00884456
 0.01965365 0.0188907  0.0133015  0.01340496 0.01400851 0.01842625
 0.01527748 0.01439687 0.01311606 0.01799388 0.01366715 0.01976257
 0.0172113  0.01115143 0.01152773 0.00962855 0.00783504 0.00897977
 0.00783178 0.01096438 0.01223009 0.008425   0.01103581 0.0105871
 0.00700039 0.00827773 0.04741375 0.02396123 0.08313222]
+----------------------+------+
|Classifier            |Result|
+---------

### Final Model

In [78]:
classifiers=[RandomForestClassifier()]

n=71
best_n_features= featureImportances.argsort()[-n:][::-1]
best_n_features= best_n_features.tolist()
vs= VectorSlicer(inputCol='features', outputCol='best_features',indices=best_n_features)
bestFeaturesDf= vs.transform(final_data)
train,test= bestFeaturesDf.randomSplit([0.7, 0.3])

columns = ['Classifier', 'Result']
vals = [('Place Holder', 'N/A')]
results = spark.createDataFrame(vals, columns)
    
for classifier in classifiers:
    new_result = ClassTrainEval(classifier, features, classes, folds, train, test)
    results = results.union(new_result)
        
results = results.where('Classifier!="Place Holder"')
results.show(100, False)
    

 
RandomForestClassifier  Feature Importances
(Scores add up to 1)
Lowest score is the least important
 
[0.03125174 0.03092625 0.0198047  0.01285609 0.00901886 0.00884094
 0.01107597 0.00604407 0.01782944 0.01784096 0.01209756 0.01036299
 0.00739588 0.0107025  0.01014866 0.00802617 0.00996827 0.00641163
 0.00950391 0.00817639 0.01140924 0.0113248  0.00877135 0.01260641
 0.00746633 0.01054556 0.00700342 0.0110766  0.00745274 0.00888207
 0.00967223 0.01454129 0.00763932 0.0119033  0.01194383 0.01215271
 0.01022457 0.01567451 0.0249122  0.01203591 0.01627062 0.01319178
 0.01913719 0.01935344 0.0118876  0.01188497 0.01117717 0.02264837
 0.01464779 0.0115643  0.01516469 0.01863266 0.01784844 0.01535855
 0.019193   0.00858562 0.00798951 0.01110997 0.00722239 0.00737345
 0.00663295 0.01139503 0.01110558 0.00857489 0.01106337 0.01121042
 0.00814962 0.00698486 0.04142901 0.02631359 0.09137778]
+----------------------+------+
|Classifier            |Result|
+----------------------+------+
|Rand

#### Make Conclusions

In [83]:
predictions = BestModel.transform(test)

In [84]:
count = predictions.filter('label!=21.0 AND prediction ==21.0').count()
print(count)
predictions.filter('label!=21.0 AND prediction ==21.0').show()

10
+-----+--------------------+--------------------+--------------------+--------------------+----------+
|label|            features|       best_features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+----------+
| 12.0|[231.375566399096...|[535.433070866141...|[0.12822352205422...|[0.00256447044108...|      21.0|
| 12.0|[500.629373568471...|[535.433070866141...|[0.54391737294963...|[0.01087834745899...|      21.0|
| 14.0|[159.800545928214...|[511.811023622047...|[0.34052287581699...|[0.00681045751633...|      21.0|
| 16.0|[245.822938778770...|[519.685039370078...|[0.02578838785735...|[5.15767757147067...|      21.0|
| 16.0|[264.866355108149...|[511.811023622047...|[0.03333333333333...|[6.66666666666666...|      21.0|
| 16.0|[343.458940732645...|[511.811023622047...|[0.11204481792717...|[0.00224089635854...|      21.0|
| 17.0|[240.533842646285...|[511.811023622047...|[0.22941176470588...|